In [1]:
"""
Author: JiaHui (Jeffrey) Lu 
Student ID: 25944800
"""
import numpy as np
import matplotlib.pyplot as plt
# used to export text file
import pandas as pd

In [2]:
def parse_coordinates(file_name):
    file = open(file_name).readlines()
    coord = []
    for item in file:
        if item[13:15] == "CA":
            coord.append([float(item[31:39]), float(item[39:47]), float(item[47:55])])
    return coord

In [3]:
A = np.array(parse_coordinates("assignment1/A.pdb"))
B = np.array(parse_coordinates("assignment1/B.pdb"))

In [4]:
def RMSD(U, V):
    U_centoid = np.sum(U, axis=0) / len(U)
    V_centoid = np.sum(V, axis=0) / len(V)
    U_prime = U - U_centoid
    V_prime = V - V_centoid
    minus = V_prime - U_prime
    plus = V_prime + U_prime
    xm = minus[:, [0]]
    ym = minus[:, [1]]
    zm = minus[:, [2]]
    xp = plus[:, [0]]
    yp = plus[:, [1]]
    zp = plus[:, [2]]
    I = np.zeros((4, 4))
    

    I[0, 0] = np.sum(xm * xm + ym * ym + zm * zm)
    I[1, 1] = np.sum(xm * xm + yp * yp + zp * zp)
    I[2, 2] = np.sum(xp * xp + ym * ym + zp * zp)
    I[3, 3] = np.sum(xp * xp + yp * yp + zm * zm)

    I[0, 1] = I[1, 0] = np.sum(yp * zm - ym * zp)
    I[0, 2] = I[2, 0] = np.sum(xm * zp - xp * zm)
    I[0, 3] = I[3, 0] = np.sum(xp * ym - xm * yp)
    I[1, 2] = I[2, 1] = np.sum(xm * ym - xp * yp)
    I[1, 3] = I[3, 1] = np.sum(xm * zm - xp * zp)
    I[2, 3] = I[3, 2] = np.sum(ym * zm - yp * zp)
    w, _ = np.linalg.eig(I)
    return np.sqrt(np.min(w)/len(U))

In [5]:
def find_well_superposable(A, B, k=9):
    count = 0
    return_array = []
    for i in range(len(A) - k + 1):
        for j in range(len(B) - k + 1):
            val = RMSD(A[i: i+k], B[j:j+k])
            if val < 1.5:
                count += 1
                return_array.append([i+1, j+1, val])
    return count, return_array

In [6]:
count, pairs = find_well_superposable(A, B)
print("the total number of well superposable pairs is:", count)

the total number of well superposable pairs is: 5219


In [7]:
df = pd.DataFrame(pairs, columns=["i", "j", "rmsd"])
df.head(10)

,i,j,rmsd
0,1,2,0.528108
1,1,55,1.369507
2,1,97,1.042116
3,1,121,0.635251
4,2,3,0.402735
5,2,19,1.462421
6,2,49,0.637293
7,2,56,1.049276
8,2,98,0.916854
9,2,122,0.306730


In [8]:
# Add the total number of pairs to the start of the file

with open("assignment1/output.txt", "w") as f:
    f.write("The total number of well-superposable fragment pairs: "+str(count)+"\n\n")

# Write the data frame to an output file
df.to_csv("assignment1/output.txt", index=None, sep=' ', float_format='%.3f', mode="a")